<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<br><b>Little Demo: Source Detection</b> <br>
Contact authors: Douglas Tucker, Melissa Graham<br>
Last verified to run: 2023-09-22 <br>
LSST Science Pipelines version: Weekly 2023_37 <br>

Rerun source detection on a processed visit image (`calexp`) and display the results.

Import packages.

In [ ]:
import matplotlib.pyplot as plt
import lsst.daf.base as dafBase
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

Set plotting parameters.

In [ ]:
plt.style.use('tableau-colorblind10')
afwDisplay.setDefaultBackend('matplotlib')

Instantiate an instance of the butler.

In [ ]:
butler = Butler(config="dp02", collections="2.2i/runs/DP0.2")

Obtain the `calexp` image for `detector` 75 in `visit` 512055.

Also obtain the sources (`src`) detected in the image with SNR $> 5$.

In [ ]:
dataId = {'visit': 512055, 'detector': 75}
calexp = butler.get('calexp', **dataId)
srcs = butler.get('src', **dataId)

In [ ]:
print(len(srcs))

Display the `calexp`.

In [ ]:
fig = plt.figure(figsize=(6, 6))
afw_display = afwDisplay.Display(fig)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

Display the `calexp` and mark the $>2000$ detected sources with SNR $>5$ with orange circles.

In [ ]:
fig = plt.figure(figsize=(6, 6))
afw_display = afwDisplay.Display(fig)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

with afw_display.Buffering():
    for s in srcs:
        afw_display.dot('o', s.getX(), s.getY(), size=20, ctype='orange')

Before redoing source detection and measurement, clear the existing `DETECTED` mask plane.

In [ ]:
calexp.mask.removeAndClearMaskPlane('DETECTED')

**Step 1: Configure the tasks.**

Create a minimal schema for the source table, and a container to record metadata.

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
algMetadata = dafBase.PropertyList()
algMetadata

Example of how to view the help for a task, or a task's configuration class.

In [ ]:
# help(charImageTask)
# help(CharacterizeImageTask.ConfigClass())

Configure the task to characterize the image properties.

In [ ]:
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask = CharacterizeImageTask(config=config)

Configure the task to detect sources in the image with SNR $> 10$.

In [ ]:
config = SourceDetectionTask.ConfigClass()
config.thresholdValue = 10
config.thresholdType = "stdev"
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

Configure the task to deblend the detected sources.

In [ ]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

Configure the task to measure the detected sources.

In [ ]:
config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=config,
                                                   algMetadata=algMetadata)

Create `SourceTable` to hold the output.

In [ ]:
tab = afwTable.SourceTable.make(schema)

**Step 2: Run the tasks.**

Characterize the image.

In [ ]:
temp = charImageTask.run(calexp)

Run source detection.

In [ ]:
result = sourceDetectionTask.run(tab, calexp)

Option to examine `result`.

A `Struct` is a generalized container for storing multiple output components and accessing them as attributes. 
Explore the content of this `Struct` with the `getDict` method.
Print the number of sources detected (about 940).

In [ ]:
# print(type(result))
# print(' ')
# for k, v in result.getDict().items():
#     print(k, type(v))
# print(' ')
# print('Number of positive peaks: ', result.numPosPeaks)

Extract the sources detected.

In [ ]:
sources = result.sources

Most of the columns for `sources` will be NaN until deblending and measurement are run.
Display the table to see the NaNs.

In [ ]:
# sources

Run deblending.

In [ ]:
sourceDeblendTask.run(calexp, sources)

Run measurement.

In [ ]:
sourceMeasurementTask.run(measCat=sources, exposure=calexp)

**Step 3: Visualize the detected sources.**

Display the `calexp` and mark the $\sim940$ detected sources with SNR $> 10$ with cyan circles.

In [ ]:
fig = plt.figure(figsize=(6, 6))
afw_display = afwDisplay.Display(fig)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

with afw_display.Buffering():
    for s in sources:
        afw_display.dot('o', s.getX(), s.getY(), size=20, ctype='cyan')

Option to save the sources to disk as a FITS table.

In [ ]:
# sources.writeFits("my_sources.fits")

**Exercise:** Rerun source detection after setting `config.thresholdValue = 3` and find about 5340 positive peaks.